In [1]:
import pandas as pd
import json
import networkx as nx
from networkx_query import search_nodes, search_edges
import numpy as np

In [2]:
options = pd.read_csv('./csv/lists/places_2020.csv')
areaType = pd.read_csv('/Users/theojolliffe/Documents/Census Data/censusAreaLookup.csv')
tfidf = pd.read_csv('/Users/theojolliffe/Documents/Wayback BBC/BBCRegionalTFIDF.csv')
typeLookup = {}
for i in areaType.index:
    typeLookup[areaType.iloc[i]['Name']]=areaType.iloc[i]['Group name']

In [121]:
areas = []
for i in options["code"]:
    try:
        areas.append(json.load(open(f'/Users/theojolliffe/Documents/Census Data/census-data-main/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
        print(i)
        pass
    
for area in areas:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']

E05009289
E05009290
E05009293
E05009294
E05009295
E05009297
E05009298
E05009299
E05009300
E05009301
E05009303
E05009306
E05009307
E05009312
E05011090


In [122]:
# Seperate the areas by area type
regions = []
for i in areas:
    if i['type']=='rgn':
        regions.append(i)
        
lads = []
for i in areas:
    if i['type']=='lad':
        lads.append(i)
        
countries = []
for i in areas:
    if (i['type']=='ew')|(i['type']=='ctry'):
        countries.append(i)

In [123]:
# Find each LAD local and national ranks and put in prioirity order
for thisLad in lads:
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_lads = []
    lad_code = thisLad['parents'][0]['code']
    for i in range(len(lads)):
        if  lad_code == lads[i]['parents'][0]['code']:
            sister_lads.append(lads[i])

    # Loop through the various data variables
    for a in thisLad['data']:
        if a in ['population', 'density', 'agemed']:
            b = 'value'
        else:
            b = 'perc'
        # Create nested object with localised rank
        thisLad['data'][a][b+"_rank_local"] = {}    
        for c in ['2011', 'change']:
            thisLad['data'][a][b+"_rank_local"][c] = {}
            for d in thisLad['data'][a][b][c]:
                vari = thisLad['data'][a][b][c][d]

                # Create sorted list of values from sister areas
                group_values = []
                for lad in sister_lads:
                    group_values.append(lad['data'][a][b][c][d])
                    group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                group_values.sort(reverse=True)

                # Find index of value of area of interest
                varRank = group_values.index(vari) + 1

                # Convert bottom half rankings into negative values
                if varRank>len(group_values)/2:
                    varRank = varRank-len(group_values)-1
                    
                natRank = thisLad['data'][a][b+"_rank"][c][d]
                if natRank > 168:
                    natRank = natRank-336-1

                thisLad['data'][a][b+"_rank_local"][c][d] = varRank

                # Append ranking data to original array
                ranks.append({
                    'label': a+'_'+b+'_'+c+'_'+d, 
                    'locRank': varRank, 
                    'natRank': natRank, 
                    'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: (abs(x['locRank']), -abs(x['value'])))
    thisLad["Priorities"] = ranks

In [124]:
topics = [{1: 'population', 2: 'value', 3: 'all'}, 
          {1: 'health', 2: 'perc', 3: 'good'},
          {1: 'travel', 2: 'perc', 3: 'car_van'},
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'economic', 2: 'perc', 3: 'employee'},
          {1: 'health', 2: 'perc', 3: 'bad'},
          {1: 'ethnicity', 2: 'perc', 3: 'black'},
          {1: 'economic', 2: 'perc', 3: 'student'},
          {1: 'economic', 2: 'perc', 3: 'self-employed'}]

ladCodes = []
for lad in lads:
    ladCodes.append(lad['code'])
    
    
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['natRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['change'][topics[j][3]]    
        df1['localRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['change'][topics[j][3]]    
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        df1['parent'].iloc[i] = lads[i]['parents'][0]['name']
    df = pd.concat([df,df1])
df['natRank'] = np.where(df['natRank'] > 168, df['natRank']-337,df['natRank'])



for topic in topics:
    for lad in lads:
        
#       Find the areas that this area has overtaken
        v2001 = lad['data'][topic[1]][topic[2]]['2001'][topic[3]]
        v2011 = lad['data'][topic[1]][topic[2]]['2011'][topic[3]]
        dfT = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]
        obje = []
        for i in range(dfT.shape[0]):
            obje.append(dfT.iloc[i]['lad'])

        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'][topic[3]] = obje
        
#       Find the area immediatly above or below
        below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]
        above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-2
        if below<0:
            below=below+1
            above=above+1
        reg = lad['parents'][0]['name']
        name_above = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[above]['lad']
        name_below = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[below]['lad']
        if above>0:
            area_above = {'name': name_above,
                         'value': df[(df['lad']==name_above)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        else:
            area_above = 'NaN'
        if below<len(df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])]):
            area_below = {'name': name_below,
                          'value': df[(df['lad']==name_below)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        else:
            area_below = 'NaN'
        
        if 'above_below' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'] = {}

        if topic[3] not in lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]] = {}
        
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["above"] = area_above
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["below"] = area_below
            
        # Add top and bottom three biggest movers for each subject to every area
        df_topic = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]
        ob = {}
        for index, row in df_topic.iterrows():
            ob[row['natRank']]= {row['lad']: row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'][topic[3]]=ob

In [250]:
# Create triple data for areas of closest proximity
geogTriples = []
for lad in lads:
    list1 = lad['bounds'][0]+lad['bounds'][1]
    for lad2 in lads:
        list2 = lad2['bounds'][0]+lad2['bounds'][1]
        listDif = [abs(list1[i]-list2[i]) for i in [0,1,2,3]]
        listDif.sort()
        if (sum(listDif[:3]) < 0.3) & (lad!=lad2):
            geogTriples.append([lad['name'], lad2['name'], ("near", round(sum(listDif[:3]), 2))])
        
nearSimilar = []
# Find nearby area of same area type
for i in geogTriples:
    try:
        if ((i[0]=="Camden")&(typeLookup[i[1]]==typeLookup["Camden"])):
            nearSimilar.append([i[0], i[1], ('near_similar', i[2][1])])
            print(i[1])
    except:
        pass
    
if len(nearSimilar)==0:
    nearSimilar = [i for i in geogTriples if i[0]=="Camden"]
    print(nearSimilar)
    
nearSimilar = sorted(nearSimilar, key=lambda x: x[2][1])

City of London
Hackney
Hammersmith and Fulham
Haringey
Islington
Kensington and Chelsea
Lambeth
Southwark
Tower Hamlets
Wandsworth
Westminster


In [251]:
nearSimilar

[['Camden', 'Islington', ('near_similar', 0.04)],
 ['Camden', 'Westminster', ('near_similar', 0.04)],
 ['Camden', 'City of London', ('near_similar', 0.09)],
 ['Camden', 'Kensington and Chelsea', ('near_similar', 0.09)],
 ['Camden', 'Hackney', ('near_similar', 0.1)],
 ['Camden', 'Hammersmith and Fulham', ('near_similar', 0.13)],
 ['Camden', 'Haringey', ('near_similar', 0.13)],
 ['Camden', 'Lambeth', ('near_similar', 0.15)],
 ['Camden', 'Wandsworth', ('near_similar', 0.15)],
 ['Camden', 'Tower Hamlets', ('near_similar', 0.17)],
 ['Camden', 'Southwark', ('near_similar', 0.23)]]

In [222]:
# Filter out priority list by subject
for lad in lads:
    subjectList = ["fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "bus", "other", "female", "male"]
    priorities = []
    priorities2011 = []
    for rank in lad['Priorities']:
        s=rank['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        if ((s[2] == "change") & (s[3] not in subjectList)):
            priorities.append(rank)
        if (s[2]!="change"):
            priorities2011.append(rank)
    lad['pri'] = priorities
    lad['pri2011'] = priorities2011

In [165]:
camden = [lad for lad in lads if lad['name']=="Camden"][0]

In [171]:
stories = []
stories=stories+[i for i in camden['pri'] if abs(i['locRank']) <= 5]
stories=stories+[i for i in camden['pri'] if abs(i['natRank']) <= 10]

In [167]:
change = sorted(camden['pri'], reverse=True, key=lambda x: abs(x['value']))
bigchange = [i for i in change if abs(i['value'])>4.5]
bigchange

[{'label': 'travel_perc_change_train_metro',
  'locRank': -3,
  'natRank': -3,
  'value': -18.18},
 {'label': 'health_perc_change_good',
  'locRank': -12,
  'natRank': 122,
  'value': 12.7},
 {'label': 'population_value_change_all',
  'locRank': -13,
  'natRank': 62,
  'value': 11.27},
 {'label': 'density_value_change_all',
  'locRank': -13,
  'natRank': 61,
  'value': 11.27},
 {'label': 'travel_perc_change_foot',
  'locRank': 4,
  'natRank': 5,
  'value': 9.58},
 {'label': 'tenure_perc_change_rented_private',
  'locRank': -7,
  'natRank': 100,
  'value': 7.12},
 {'label': 'ethnicity_perc_change_white',
  'locRank': 8,
  'natRank': -63,
  'value': -6.85},
 {'label': 'travel_perc_change_car_van',
  'locRank': 3,
  'natRank': -66,
  'value': -4.95}]

In [168]:
for i in range(len(bigchange)):
    if len(stories)<4:
        stories=stories+[bigchange[i]]

In [169]:
smallchange = sorted([i for i in change if abs(i['value'])<1], reverse=False, key=lambda x: abs(x['value']))
smallchange

[{'label': 'ethnicity_perc_change_black',
  'locRank': -6,
  'natRank': -6,
  'value': -0.12},
 {'label': 'age10yr_perc_change_10-19',
  'locRank': 14,
  'natRank': 83,
  'value': -0.17},
 {'label': 'age10yr_perc_change_0-9',
  'locRank': -12,
  'natRank': 78,
  'value': -0.2},
 {'label': 'age10yr_perc_change_70plus',
  'locRank': 5,
  'natRank': -94,
  'value': -0.22},
 {'label': 'age10yr_perc_change_30-39',
  'locRank': 12,
  'natRank': 19,
  'value': -0.25},
 {'label': 'economic_perc_change_unemployed',
  'locRank': -2,
  'natRank': -4,
  'value': -0.48},
 {'label': 'age10yr_perc_change_50-59',
  'locRank': -8,
  'natRank': 166,
  'value': -0.56},
 {'label': 'economic_perc_change_student',
  'locRank': -8,
  'natRank': 149,
  'value': 0.61},
 {'label': 'age10yr_perc_change_20-29',
  'locRank': -4,
  'natRank': -15,
  'value': -0.93}]

In [133]:
for i in range(len(smallchange)):
    if len(stories)<5:
        stories=stories+[smallchange[i]]

In [194]:
sorted([(i, camden['data']['age10yr']['value']['2011'][i]-camden['data']['age10yr']['value']['2001'][i]) for i in camden['data']['age10yr']['value']['2001'].keys()], 
       reverse=True, key=lambda x: x[1])

[('all', 22282),
 ('40-49', 5327),
 ('60-69', 3936),
 ('30-39', 3909),
 ('20-29', 2976),
 ('0-9', 2046),
 ('10-19', 1895),
 ('70plus', 1243),
 ('50-59', 950)]

In [245]:
camden['data']['health']['perc']['2011']

{'bad': 5.6, 'fair': 10.42, 'good': 83.97}

In [136]:
nearSimilarData['data']['population']['value']['2011']

{'all': 206125, 'female': 104720, 'male': 101405}

In [246]:
london['data']['health']['perc']['2011']

{'bad': 4.96, 'fair': 11.19, 'good': 83.84}

In [244]:
countries[1]['data']['health']['perc']['2011']

{'bad': 5.49, 'fair': 13.12, 'good': 81.39}

In [172]:
stories

[{'label': 'agemed_value_change_all',
  'locRank': 1,
  'natRank': 94,
  'value': 1.0},
 {'label': 'tenure_perc_change_owned',
  'locRank': 2,
  'natRank': 60,
  'value': -2.54},
 {'label': 'economic_perc_change_unemployed',
  'locRank': -2,
  'natRank': -4,
  'value': -0.48},
 {'label': 'travel_perc_change_train_metro',
  'locRank': -3,
  'natRank': -3,
  'value': -18.18},
 {'label': 'travel_perc_change_car_van',
  'locRank': 3,
  'natRank': -66,
  'value': -4.95},
 {'label': 'travel_perc_change_home',
  'locRank': 3,
  'natRank': 3,
  'value': 3.91},
 {'label': 'travel_perc_change_foot',
  'locRank': 4,
  'natRank': 5,
  'value': 9.58},
 {'label': 'age10yr_perc_change_20-29',
  'locRank': -4,
  'natRank': -15,
  'value': -0.93},
 {'label': 'age10yr_perc_change_70plus',
  'locRank': 5,
  'natRank': -94,
  'value': -0.22},
 {'label': 'economic_perc_change_unemployed',
  'locRank': -2,
  'natRank': -4,
  'value': -0.48},
 {'label': 'travel_perc_change_train_metro',
  'locRank': -3,
  'n

In [223]:
camden['pri2011']

[{'label': 'economic_perc_2011_employee',
  'locRank': -2,
  'natRank': -20,
  'value': 46.41},
 {'label': 'economic_perc_2011_inactive',
  'locRank': 3,
  'natRank': 95,
  'value': 31.89},
 {'label': 'travel_perc_2011_foot',
  'locRank': 4,
  'natRank': 30,
  'value': 16.19},
 {'label': 'travel_perc_2011_car_van',
  'locRank': -4,
  'natRank': -4,
  'value': 10.73},
 {'label': 'tenure_perc_2011_rent_free',
  'locRank': 4,
  'natRank': 32,
  'value': 1.81},
 {'label': 'travel_perc_2011_other',
  'locRank': 4,
  'natRank': 20,
  'value': 1.0},
 {'label': 'tenure_perc_2011_shared_ownership',
  'locRank': -4,
  'natRank': 151,
  'value': 0.69},
 {'label': 'age10yr_perc_2011_40-49',
  'locRank': -5,
  'natRank': -24,
  'value': 13.26},
 {'label': 'age10yr_perc_2011_0-9',
  'locRank': -5,
  'natRank': -127,
  'value': 10.92},
 {'label': 'travel_perc_2011_home',
  'locRank': 5,
  'natRank': 54,
  'value': 8.24},
 {'label': 'tenure_perc_2011_rented_private',
  'locRank': 6,
  'natRank': 6,
  

In [156]:
london=[i for i in regions if i['name']=="London"][0]

In [81]:
def reg(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    region=[i for i in regions if i['name']==camden['parents'][0]['name']][0]
    return region['data'][s[0]][s[1]][s[2]][s[3]]
regDiff = [(i['label'], i['value']-reg(i)) for i in change]
sorted(regDiff, reverse=True, key=lambda x: abs(x[1]))

[('travel_perc_change_train_metro', -19.8),
 ('travel_perc_change_foot', 8.15),
 ('travel_perc_change_home', 6.35),
 ('agemed_value_change_all', 5.8),
 ('tenure_perc_change_owned', 5.18),
 ('ethnicity_perc_change_white', 4.51),
 ('population_value_change_all', -2.700000000000001),
 ('tenure_perc_change_rented_private', -2.62),
 ('ethnicity_perc_change_black', -2.52),
 ('tenure_perc_change_rented_social', -2.1300000000000003),
 ('economic_perc_change_employee', 1.77),
 ('travel_perc_change_car_van', 1.7199999999999998),
 ('economic_perc_change_unemployed', -1.33),
 ('ethnicity_perc_change_asian', -1.29),
 ('economic_perc_change_inactive', 0.8400000000000003),
 ('economic_perc_change_self-employed', -0.7800000000000002),
 ('economic_perc_change_student', -0.4900000000000001),
 ('health_perc_change_good', -0.3200000000000003),
 ('health_perc_change_bad', -0.20999999999999996),
 ('ethnicity_perc_change_mixed', 0.020000000000000018)]

In [83]:
def cou(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return countries[1]['data'][s[0]][s[1]][s[2]][s[3]]
couDiff = [(i['label'], i['value']-cou(i)) for i in change]
sorted(couDiff, reverse=True, key=lambda x: abs(x[1]))

[('travel_perc_change_train_metro', -20.22),
 ('travel_perc_change_foot', 8.81),
 ('travel_perc_change_home', 7.73),
 ('travel_perc_change_car_van', -5.87),
 ('population_value_change_all', 3.3899999999999997),
 ('tenure_perc_change_rented_social', -2.5700000000000003),
 ('tenure_perc_change_owned', 2.3899999999999997),
 ('economic_perc_change_employee', 1.53),
 ('economic_perc_change_unemployed', -1.51),
 ('ethnicity_perc_change_white', -1.3399999999999999),
 ('ethnicity_perc_change_black', -1.2999999999999998),
 ('ethnicity_perc_change_asian', 1.2000000000000002),
 ('ethnicity_perc_change_mixed', 0.8800000000000001),
 ('economic_perc_change_self-employed', 0.45999999999999996),
 ('economic_perc_change_student', -0.25),
 ('agemed_value_change_all', 0.23999999999999977),
 ('economic_perc_change_inactive', -0.22999999999999998),
 ('tenure_perc_change_rented_private', 0.16999999999999993),
 ('health_perc_change_good', 0.06999999999999851),
 ('health_perc_change_bad', 0.010000000000000231

In [92]:
nearSimilarData = [lad for lad in lads if lad['name']==nearSimilar[0][1]][0]
print(camden['name'], " population: ", camden['data']['population']['value']['2011']['all'])
print(nearSimilarData['name'], " population: ", nearSimilarData['data']['population']['value']['2011']['all'])
difference = camden['data']['population']['value']['2011']['all']- nearSimilarData['data']['population']['value']['2011']['all']
print("Difference: ", difference, ", ", round(100*difference/camden['data']['population']['value']['2011']['all']), "%")

Camden  population:  220338
Islington  population:  206125
Difference:  14213 ,  6 %


In [93]:
def near(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return nearSimilarData['data'][s[0]][s[1]][s[2]][s[3]]
nearDiff = [(i['label'], i['value']-near(i)) for i in change]
sorted(nearDiff, reverse=True, key=lambda x: abs(x[1]))

[('travel_perc_change_car_van', 6.1000000000000005),
 ('agemed_value_change_all', 5.88),
 ('population_value_change_all', -5.809999999999999),
 ('tenure_perc_change_rented_private', -3.2800000000000002),
 ('travel_perc_change_home', 3.23),
 ('travel_perc_change_train_metro', -3.209999999999999),
 ('tenure_perc_change_rented_social', 2.6399999999999997),
 ('economic_perc_change_inactive', 2.5500000000000003),
 ('economic_perc_change_employee', -2.19),
 ('ethnicity_perc_change_asian', 1.8900000000000001),
 ('health_perc_change_good', -1.6600000000000001),
 ('travel_perc_change_foot', 1.25),
 ('ethnicity_perc_change_black', -1.03),
 ('tenure_perc_change_owned', 0.8799999999999999),
 ('health_perc_change_bad', 0.8199999999999998),
 ('ethnicity_perc_change_mixed', -0.55),
 ('ethnicity_perc_change_white', 0.35000000000000053),
 ('economic_perc_change_student', -0.20999999999999996),
 ('economic_perc_change_unemployed', -0.10999999999999999),
 ('economic_perc_change_self-employed', -0.0500000

In [95]:
sorted([(lad['name'], lad['pri'][0]) for lad in lads if lad['name'] in [i[1] for i in nearSimilar if i[0]=="Camden"]], key=lambda x: abs(x[1]['locRank']))

[('City of London',
  {'label': 'travel_perc_change_train_metro',
   'locRank': -1,
   'natRank': -1,
   'value': -56.44}),
 ('Hackney',
  {'label': 'ethnicity_perc_change_white',
   'locRank': 1,
   'natRank': -99,
   'value': -4.73}),
 ('Haringey',
  {'label': 'travel_perc_change_car_van',
   'locRank': -1,
   'natRank': -1,
   'value': -25.96}),
 ('Kensington and Chelsea',
  {'label': 'health_perc_change_good',
   'locRank': -1,
   'natRank': -24,
   'value': 11.14}),
 ('Southwark',
  {'label': 'tenure_perc_change_owned',
   'locRank': 1,
   'natRank': 33,
   'value': -1.62}),
 ('Tower Hamlets',
  {'label': 'population_value_change_all',
   'locRank': 1,
   'natRank': 1,
   'value': 29.57}),
 ('Westminster',
  {'label': 'travel_perc_change_car_van',
   'locRank': 1,
   'natRank': -98,
   'value': -1.92}),
 ('Lambeth',
  {'label': 'ethnicity_perc_change_mixed',
   'locRank': 2,
   'natRank': 3,
   'value': 2.84}),
 ('Islington',
  {'label': 'tenure_perc_change_rented_social',
   'loc

In [195]:
equalAgeChange = [lad['name'] for 
 lad in lads if (lad['parents'][0]['name'] == camden['parents'][0]['name']) 
 & (lad['data']['agemed']['value']['change']['all']==camden['data']['agemed']['value']['change']['all'])]

In [196]:
len(equalAgeChange)

4